Задание 1: Эхо-сервер (TCP)

Файл: tcp_echo_server.py (TCP Эхо-сервер)

In [3]:
# tcp_echo_server.py
import socket
import time # Для измерения времени на стороне клиента (не сервера)

def run_echo_server(host='127.0.0.1', port=65432):
    """
    Запускает TCP эхо-сервер.
    """
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        try:
            s.bind((host, port))
            s.listen()
            print(f"[Сервер] Эхо-сервер запущен и слушает на {host}:{port}")

            while True: # Основной цикл для приема новых подключений
                conn, addr = s.accept() # Принимаем новое подключение
                with conn: # Используем 'with' для автоматического закрытия conn
                    print(f"[Сервер] Установлено соединение с {addr}")
                    
                    data = conn.recv(1024) # Получаем данные от клиента (до 1024 байт)
                    if not data:
                        # Если клиент закрыл соединение без отправки данных (маловероятно для этого примера)
                        print(f"[Сервер] Клиент {addr} закрыл соединение без данных.")
                        # break # Можно раскомментировать, если сервер должен обслуживать только одного клиента
                        continue # Готовы к новому подключению

                    decoded_data = data.decode('utf-8')
                    print(f"[Сервер] Получено от {addr}: '{decoded_data}'")
                    
                    # Отправляем данные обратно клиенту (эхо)
                    conn.sendall(data) 
                    print(f"[Сервер] Отправлено обратно клиенту {addr}: '{decoded_data}'")
                    
                print(f"[Сервер] Соединение с {addr} завершено.")
                # В данном задании сервер завершает соединение после одного ответа.
                # Если нужно обрабатывать несколько запросов от одного клиента в рамках одного соединения,
                # цикл приема данных (recv) должен быть внутри 'with conn:'.
                # Здесь, после 'with conn:', соединение conn закрывается, и сервер готов к новому accept().
                # Для обработки только одного клиента и завершения работы сервера после этого,
                # можно было бы убрать внешний 'while True' и завершить работу после первого 'with conn:'.
                # Но обычно сервер продолжает слушать.
                # В данном задании "Завершает соединение после ответа" подразумевает, что каждое соединение - это один эхо-обмен.
        except OSError as e:
            print(f"[Сервер] Ошибка сокета: {e}")
        except KeyboardInterrupt:
            print("[Сервер] Сервер остановлен вручную.")
        finally:
            print("[Сервер] Эхо-сервер завершает работу.")

if __name__ == "__main__":
    run_echo_server()

[Сервер] Эхо-сервер запущен и слушает на 127.0.0.1:65432
[Сервер] Сервер остановлен вручную.
[Сервер] Эхо-сервер завершает работу.


Файл: tcp_echo_client.py (TCP Эхо-клиент)

In [ ]:
# tcp_echo_client.py
import socket
import time

def run_echo_client(host='127.0.0.1', port=65432, message="Привет, сервер!"):
    """
    Запускает TCP эхо-клиент.
    """
    start_time = 0
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            print(f"[Клиент] Попытка подключения к серверу {host}:{port}...")
            start_time = time.perf_counter() # Время перед подключением
            s.connect((host, port))
            connect_time = time.perf_counter()
            print(f"[Клиент] Успешно подключен к серверу.")

            print(f"[Клиент] Отправка сообщения: '{message}'")
            s.sendall(message.encode('utf-8')) # Отправляем сообщение серверу

            data = s.recv(1024) # Получаем ответ от сервера
            end_time = time.perf_counter() # Время после получения ответа

            if data:
                response = data.decode('utf-8')
                print(f"[Клиент] Получен ответ от сервера: '{response}'")
            else:
                print("[Клиент] Сервер закрыл соединение без ответа или ответ пустой.")

            # Измерение времени
            connection_duration = connect_time - start_time
            round_trip_time = end_time - connect_time # Время от отправки (примерно) до получения ответа
            total_operation_time = end_time - start_time # Общее время работы клиента

            print(f"\n[Клиент] Статистика времени:")
            print(f"  Время на установку соединения: {connection_duration*1000:.2f} мс")
            # round_trip_time здесь не совсем точен, т.к. connect_time - это время после s.connect(),
            # а не перед s.sendall(). Для более точного RTT, нужно засекать время вокруг sendall и recv.
            # Давайте измерим RTT более точно:
            
            # Повторная отправка для более точного RTT (если сервер обрабатывает несколько запросов)
            # Но наш текущий сервер закрывает соединение после первого ответа.
            # Поэтому измерим общее время.
            print(f"  Общее время работы клиента (от подключения до получения ответа): {total_operation_time*1000:.2f} мс")

    except socket.error as e:
        print(f"[Клиент] Ошибка сокета: {e}")
        if start_time > 0: # Если подключение не удалось, но мы начали замер
            failed_connect_time = time.perf_counter()
            print(f"  Время до ошибки подключения: {(failed_connect_time - start_time)*1000:.2f} мс")
    except Exception as e:
        print(f"[Клиент] Произошла ошибка: {e}")
    finally:
        print("[Клиент] Клиент завершает работу.")


if __name__ == "__main__":
    # Пример использования
    client_message = input("Введите сообщение для отправки серверу (нажмите Enter для 'Привет, мир!'): ")
    if not client_message:
        client_message = "Привет, мир!"
    run_echo_client(message=client_message)

    # Пример из задания:
    # print("\n--- Пример из задания ---")
    # run_echo_client(message="Привет, сервер!")

[Клиент] Попытка подключения к серверу 127.0.0.1:65432...
[Клиент] Ошибка сокета: [Errno 111] Connection refused
  Время до ошибки подключения: 0.23 мс
[Клиент] Клиент завершает работу.


Файл: tcp_chat_server_threaded.py (Многопоточный TCP Чат-сервер)

In [4]:
# tcp_chat_server_threaded.py
import socket
import threading

class ChatServer:
    """
    Многопоточный TCP чат-сервер.
    """
    def __init__(self, host='127.0.0.1', port=65433):
        self.host = host
        self.port = port
        self.server_socket = None
        self.clients = {}  # Словарь для хранения подключенных клиентов {conn: username}
        self.client_locks = {} # Словарь для блокировок сокетов клиентов {conn: threading.Lock()}
        self.broadcast_lock = threading.Lock() # Блокировка для безопасной отправки сообщений всем
        print("[Сервер Чата] Инициализация сервера...")

    def broadcast(self, message: str, sender_conn: socket.socket = None, sender_username: str = "Сервер"):
        """Отправляет сообщение всем подключенным клиентам, кроме отправителя."""
        with self.broadcast_lock: # Защищаем clients от изменения во время итерации
            # Создаем копию ключей, чтобы избежать проблем с изменением словаря во время итерации
            # (хотя удаление происходит в handle_client после выхода из этой функции)
            clients_to_send = list(self.clients.keys()) 
            
            for client_conn in clients_to_send:
                if client_conn != sender_conn: # Не отправляем сообщение самому себе
                    try:
                        # Используем индивидуальную блокировку для сокета клиента
                        # if client_conn in self.client_locks: # Проверка на случай, если клиент уже отключается
                        #     with self.client_locks[client_conn]:
                        #         client_conn.sendall(message.encode('utf-8'))
                        # Вместо блокировки, просто пытаемся отправить. Если ошибка - клиент отключился.
                        client_conn.sendall(message.encode('utf-8'))
                    except (socket.error, BrokenPipeError, ConnectionResetError) as e:
                        print(f"[Сервер Чата] Ошибка отправки клиенту {self.clients.get(client_conn, 'Неизвестный')}: {e}. Удаление клиента.")
                        # Удаление клиента должно происходить в handle_client или в основном цикле
                        # Здесь просто ловим ошибку. Удаление произойдет позже.
                        # self._remove_client(client_conn) # Небезопасно изменять словарь во время итерации без доп. мер
                        pass # Позволим handle_client обработать отключение

    def send_private_message(self, recipient_username: str, message: str, sender_username: str):
        """Отправляет приватное сообщение указанному пользователю."""
        with self.broadcast_lock: # Защита словаря clients
            recipient_conn = None
            for conn, uname in self.clients.items():
                if uname == recipient_username:
                    recipient_conn = conn
                    break
            
            if recipient_conn:
                try:
                    formatted_message = f"[Приватное от {sender_username}]: {message}\n"
                    recipient_conn.sendall(formatted_message.encode('utf-8'))
                    print(f"[Сервер Чата] Приватное сообщение от {sender_username} для {recipient_username} отправлено.")
                    return True
                except (socket.error, BrokenPipeError, ConnectionResetError) as e:
                    print(f"[Сервер Чата] Ошибка отправки приватного сообщения {recipient_username}: {e}")
                    return False
            else:
                print(f"[Сервер Чата] Пользователь {recipient_username} не найден для приватного сообщения.")
                # Можно отправить отправителю уведомление, что пользователь не найден
                sender_conn_obj = None
                for conn, uname in self.clients.items():
                    if uname == sender_username:
                        sender_conn_obj = conn
                        break
                if sender_conn_obj:
                    try:
                        sender_conn_obj.sendall(f"[Сервер]: Пользователь '{recipient_username}' не найден или оффлайн.\n".encode('utf-8'))
                    except: pass # Игнорируем ошибки при отправке уведомления отправителю
                return False


    def _remove_client(self, conn: socket.socket):
        """Безопасное удаление клиента из списков."""
        with self.broadcast_lock: # Защищаем словари
            username = self.clients.pop(conn, None)
            self.client_locks.pop(conn, None) # Удаляем и блокировку, если она была
            if username:
                print(f"[Сервер Чата] Клиент {username} ({conn.getpeername() if conn.fileno() != -1 else 'N/A'}) отключился.")
                self.broadcast(f"[Сервер]: Пользователь {username} покинул чат.\n", sender_conn=None) # Уведомляем всех
            try:
                conn.close()
            except socket.error:
                pass # Сокет мог уже быть закрыт


    def handle_client(self, conn: socket.socket, addr):
        """Обрабатывает подключение клиента в отдельном потоке."""
        print(f"[Сервер Чата] Новое подключение от {addr}")
        username = ""
        try:
            # Запрос имени пользователя
            conn.sendall("Введите ваше имя пользователя: ".encode('utf-8'))
            username_data = conn.recv(1024)
            if not username_data:
                print(f"[Сервер Чата] Клиент {addr} отключился не представившись.")
                return

            username = username_data.decode('utf-8').strip()
            if not username: # Если имя пустое после strip
                 username = f"User{random.randint(1000,9999)}" # Генерируем случайное имя
                 conn.sendall(f"[Сервер]: Ваше имя установлено как '{username}'.\n".encode('utf-8'))


            # Проверка на уникальность имени (упрощенная, могут быть гонки)
            # Для надежности нужна блокировка при проверке и добавлении
            with self.broadcast_lock:
                while username in self.clients.values():
                    conn.sendall(f"Имя '{username}' уже занято. Введите другое имя: ".encode('utf-8'))
                    username_data = conn.recv(1024)
                    if not username_data: return # Клиент отвалился
                    username = username_data.decode('utf-8').strip()
                    if not username: username = f"User{random.randint(1000,9999)}"
                
                self.clients[conn] = username
                # self.client_locks[conn] = threading.Lock() # Если будем использовать индивидуальные блокировки

            print(f"[Сервер Чата] Пользователь {username} ({addr}) присоединился к чату.")
            conn.sendall(f"Добро пожаловать в чат, {username}!\nВведите 'exit' для выхода или '@имя_пользователя: сообщение' для приватного сообщения.\n".encode('utf-8'))
            self.broadcast(f"[Сервер]: {username} присоединился к чату.\n", sender_conn=conn, sender_username=username)

            while True:
                data = conn.recv(1024)
                if not data: # Соединение закрыто клиентом
                    break 
                
                message = data.decode('utf-8').strip()
                if message.lower() == 'exit':
                    break

                if message.startswith('@'):
                    parts = message.split(':', 1)
                    if len(parts) == 2:
                        recipient = parts[0][1:].strip() # Убираем @ и пробелы
                        private_msg_content = parts[1].strip()
                        if recipient and private_msg_content:
                            if not self.send_private_message(recipient, private_msg_content, username):
                                conn.sendall(f"[Сервер]: Не удалось отправить приватное сообщение пользователю '{recipient}'.\n".encode('utf-8'))
                            else:
                                # Уведомление отправителю об успешной отправке (опционально)
                                conn.sendall(f"[Вы -> {recipient}]: {private_msg_content}\n".encode('utf-8'))

                        else:
                            conn.sendall("[Сервер]: Неверный формат приватного сообщения. Используйте @имя: сообщение\n".encode('utf-8'))
                    else:
                        conn.sendall("[Сервер]: Неверный формат приватного сообщения. Используйте @имя: сообщение\n".encode('utf-8'))
                else:
                    broadcast_message = f"[{username}]: {message}\n"
                    print(f"[Сервер Чата] Сообщение от {username}: {message}")
                    self.broadcast(broadcast_message, sender_conn=conn, sender_username=username)

        except (socket.error, ConnectionResetError, BrokenPipeError) as e:
            print(f"[Сервер Чата] Ошибка соединения с {username if username else addr}: {e}")
        except Exception as e_gen:
            print(f"[Сервер Чата] Непредвиденная ошибка с {username if username else addr}: {e_gen}")
        finally:
            self._remove_client(conn)


    def start(self):
        """Запускает сервер чата."""
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        # Позволяет повторно использовать адрес сразу после закрытия сервера
        self.server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) 
        try:
            self.server_socket.bind((self.host, self.port))
            self.server_socket.listen(5) # До 5 ожидающих подключений
            print(f"[Сервер Чата] Сервер запущен и слушает на {self.host}:{self.port}")

            while True:
                conn, addr = self.server_socket.accept()
                # Создаем новый поток для каждого клиента
                client_thread = threading.Thread(target=self.handle_client, args=(conn, addr))
                client_thread.daemon = True # Потоки завершатся, если основной поток завершится
                client_thread.start()
        except OSError as e:
            print(f"[Сервер Чата] Ошибка сокета при запуске: {e}")
        except KeyboardInterrupt:
            print("[Сервер Чата] Сервер остановлен вручную.")
        finally:
            if self.server_socket:
                self.server_socket.close()
            # Закрываем все клиентские соединения при остановке сервера
            with self.broadcast_lock:
                for conn in list(self.clients.keys()): # list() для копии ключей
                    self._remove_client(conn)
            print("[Сервер Чата] Сервер завершил работу.")

if __name__ == "__main__":
    chat_server = ChatServer()
    chat_server.start()

[Сервер Чата] Инициализация сервера...
[Сервер Чата] Сервер запущен и слушает на 127.0.0.1:65433
[Сервер Чата] Сервер остановлен вручную.
[Сервер Чата] Сервер завершил работу.


Файл: tcp_chat_client_threaded.py (TCP Чат-клиент)

In [5]:
# tcp_chat_client_threaded.py
import socket
import threading
import sys # Для sys.stdout.write и flush

class ChatClient:
    def __init__(self, host='127.0.0.1', port=65433):
        self.host = host
        self.port = port
        self.client_socket = None
        self._stop_event = threading.Event() # Событие для остановки потока приема

    def receive_messages(self):
        """Поток для приема сообщений от сервера."""
        while not self._stop_event.is_set():
            try:
                if self.client_socket:
                    # Устанавливаем таймаут, чтобы не блокироваться навечно и проверять _stop_event
                    self.client_socket.settimeout(1.0) 
                    data = self.client_socket.recv(4096) # Буфер побольше для сообщений чата
                    if data:
                        message = data.decode('utf-8')
                        # Печатаем сообщение и сразу же выводим приглашение ввода, если оно было стерто
                        sys.stdout.write("\r" + " " * 60 + "\r") # Очищаем текущую строку (примерно)
                        sys.stdout.write(message) # Печатаем сообщение от сервера
                        sys.stdout.write("Вы: ") # Восстанавливаем приглашение ввода
                        sys.stdout.flush() # Принудительно выводим буфер
                    else:
                        # Сервер закрыл соединение
                        print("\r[Клиент Чата] Соединение с сервером потеряно.")
                        self._stop_event.set() # Сигнализируем основному потоку о необходимости завершения
                        break
            except socket.timeout:
                continue # Таймаут, просто проверяем _stop_event и продолжаем цикл
            except (socket.error, ConnectionResetError, BrokenPipeError) as e:
                print(f"\r[Клиент Чата] Ошибка получения сообщения: {e}")
                self._stop_event.set()
                break
            except Exception as e_gen:
                print(f"\r[Клиент Чата] Непредвиденная ошибка при получении: {e_gen}")
                self._stop_event.set()
                break
        print("\r[Клиент Чата] Поток приема сообщений завершен.")


    def start(self):
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            self.client_socket.connect((self.host, self.port))
            print("[Клиент Чата] Успешно подключен к чат-серверу.")

            # Запускаем поток для приема сообщений
            receive_thread = threading.Thread(target=self.receive_messages)
            receive_thread.daemon = True
            receive_thread.start()

            # Основной цикл для отправки сообщений
            while not self._stop_event.is_set():
                # sys.stdout.write("Вы: ") # Приглашение к вводу
                # sys.stdout.flush()
                message_to_send = input() # Блокирующий ввод
                
                if self._stop_event.is_set(): # Проверяем после input, если сервер отвалился во время ожидания ввода
                    break

                if message_to_send:
                    try:
                        self.client_socket.sendall(message_to_send.encode('utf-8'))
                        if message_to_send.lower() == 'exit':
                            print("[Клиент Чата] Отправка команды 'exit', отключение...")
                            self._stop_event.set() # Сигнализируем потоку приема о завершении
                            break 
                    except (socket.error, BrokenPipeError) as e:
                        print(f"[Клиент Чата] Ошибка отправки сообщения: {e}")
                        self._stop_event.set()
                        break
            
            # Ждем завершения потока приема, если он еще работает
            if receive_thread.is_alive():
                 print("[Клиент Чата] Ожидание завершения потока приема...")
                 receive_thread.join(timeout=2.0) # Даем немного времени на завершение

        except socket.error as e:
            print(f"[Клиент Чата] Ошибка сокета при подключении: {e}")
        except KeyboardInterrupt:
            print("\r[Клиент Чата] Отключение по команде пользователя (Ctrl+C)...")
            self._stop_event.set()
            if self.client_socket:
                try:
                    self.client_socket.sendall("exit".encode('utf-8')) # Попытка уведомить сервер
                except: pass
        finally:
            self._stop_event.set() # Убеждаемся, что событие установлено
            if self.client_socket:
                self.client_socket.close()
            print("[Клиент Чата] Клиент завершил работу.")

if __name__ == "__main__":
    chat_client = ChatClient()
    chat_client.start()

[Клиент Чата] Ошибка сокета при подключении: [Errno 111] Connection refused
[Клиент Чата] Клиент завершил работу.


Задание 3: Сервер времени (UDP)

In [9]:
# udp_time_server.py
import socket
import datetime
import time

def run_udp_time_server(host='127.0.0.1', port=65434):
    """
    Запускает UDP сервер времени.
    """
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s: # SOCK_DGRAM для UDP
        try:
            s.bind((host, port))
            print(f"[UDP Сервер Времени] Сервер запущен и слушает на {host}:{port}")

            while True:
                # Принимаем данные и адрес отправителя
                data, addr = s.recvfrom(1024) # Для UDP используется recvfrom
                
                decoded_data = data.decode('utf-8').strip()
                print(f"[UDP Сервер Времени] Получен запрос '{decoded_data}' от {addr}")

                if decoded_data.lower() == "time":
                    current_time_str = datetime.datetime.now().strftime("%H:%M:%S")
                    s.sendto(current_time_str.encode('utf-8'), addr) # Отправляем ответ конкретному адресу
                    print(f"[UDP Сервер Времени] Отправлено время '{current_time_str}' клиенту {addr}")
                else:
                    error_message = "Неизвестный запрос. Отправьте 'time' для получения времени."
                    s.sendto(error_message.encode('utf-8'), addr)
                    print(f"[UDP Сервер Времени] Отправлен ответ об ошибке клиенту {addr}")

        except OSError as e:
            print(f"[UDP Сервер Времени] Ошибка сокета: {e}")
        except KeyboardInterrupt:
            print("[UDP Сервер Времени] Сервер остановлен вручную.")
        finally:
            print("[UDP Сервер Времени] Сервер времени завершает работу.")

if __name__ == "__main__":
    run_udp_time_server()

[UDP Сервер Времени] Сервер запущен и слушает на 127.0.0.1:65434
[UDP Сервер Времени] Сервер остановлен вручную.
[UDP Сервер Времени] Сервер времени завершает работу.


Файл: udp_time_client.py (UDP Клиент Времени)

In [ ]:
# udp_time_client.py
import socket
import time

def run_udp_time_client(host='127.0.0.1', port=65434):
    """
    Запускает UDP клиент времени.
    """
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
        try:
            # Для UDP connect() не обязателен, но может использоваться для "фиксации" адресата
            # s.connect((host, port)) # Если бы использовали connect, то send/recv вместо sendto/recvfrom

            request_message = "time"
            
            print(f"[UDP Клиент Времени] Отправка запроса '{request_message}' на сервер {host}:{port}")
            start_time = time.perf_counter() # Время перед отправкой
            
            s.sendto(request_message.encode('utf-8'), (host, port))

            # Устанавливаем таймаут для ожидания ответа от UDP сервера
            s.settimeout(5.0) # Ждать ответа не более 5 секунд

            try:
                data, server_addr = s.recvfrom(1024) # Получаем данные и адрес сервера
                end_time = time.perf_counter() # Время после получения ответа
                
                response = data.decode('utf-8')
                print(f"[UDP Клиент Времени] Получен ответ от сервера {server_addr}: '{response}'")

                # Измерение задержки
                round_trip_delay = (end_time - start_time) * 1000 # в миллисекундах
                print(f"[UDP Клиент Времени] Задержка (Round-Trip Time): {round_trip_delay:.2f} мс")

            except socket.timeout:
                print("[UDP Клиент Времени] Таймаут: Ответ от сервера не получен в течение 5 секунд.")
            except socket.error as e_recv:
                 print(f"[UDP Клиент Времени] Ошибка при получении ответа: {e_recv}")


        except socket.error as e:
            print(f"[UDP Клиент Времени] Ошибка сокета: {e}")
        except Exception as e_gen:
            print(f"[UDP Клиент Времени] Произошла непредвиденная ошибка: {e_gen}")
        finally:
            print("[UDP Клиент Времени] Клиент времени завершает работу.")


if __name__ == "__main__":
    run_udp_time_client()

[UDP Клиент Времени] Отправка запроса 'time' на сервер 127.0.0.1:65434
[UDP Клиент Времени] Таймаут: Ответ от сервера не получен в течение 5 секунд.
[UDP Клиент Времени] Клиент времени завершает работу.
